In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path
from datetime import datetime

# 🧠 Configuración del entorno y cliente
load_dotenv()
client = OpenAI()
RAIZ = "."
EXCLUIR_DIRS = {"venv", ".git", "__pycache__"}
EXCLUIR_ARCHIVOS = {"requirements.txt", ".env", ".env.example", "README.md", "Dockerfile"}
EXT_VALIDAS = {".py", ".sh", ".md", ".txt"}

# 📂 Leer estructura base si existe
estructura_base = ""
estructura_file = Path("estructura_base.txt")
if estructura_file.exists():
    estructura_base = estructura_file.read_text()

# 📄 Preparar log
log_path = Path("logs/archivos_generados.txt")
log_path.parent.mkdir(exist_ok=True)
log = []

def es_valido(ruta):
    nombre = os.path.basename(ruta)
    ext = os.path.splitext(nombre)[1]
    partes = Path(ruta).parts
    if any(p in EXCLUIR_DIRS for p in partes):
        return False
    if nombre in EXCLUIR_ARCHIVOS:
        return False
    if ext not in EXT_VALIDAS:
        return False
    if os.path.getsize(ruta) > 10:
        return False
    return True

def limpiar_codigo(contenido):
    if contenido.startswith("```"):
        contenido = contenido.strip("`").strip()
        if "\n" in contenido:
            contenido = contenido.split("\n", 1)[1]
    return contenido.strip()

def generar_contenido(nombre_archivo):
    prompt = f"""
Eres un programador experto. Escribe el contenido adecuado para '{nombre_archivo.name}' 
como parte de un proyecto Python moderno, profesional y modular.

Estructura del proyecto:
{estructura_base}

No des explicaciones. Solo devuelve el contenido del archivo.
"""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return limpiar_codigo(response.choices[0].message.content)

# 🚀 Proceso principal
for root, _, files in os.walk(RAIZ):
    if any(ex in root for ex in EXCLUIR_DIRS):
        continue
    for file in files:
        ruta = os.path.join(root, file)
        if es_valido(ruta):
            print(f"✍️ Generando para: {ruta}")
            contenido = generar_contenido(Path(ruta))
            with open(ruta, "w", encoding="utf-8") as f:
                f.write(contenido)
            log.append(f"{datetime.now().isoformat()} - {ruta}")

# 📝 Guardar log
with open(log_path, "a", encoding="utf-8") as log_file:
    log_file.write("\n".join(log) + "\n")

print(f"✅ Finalizado. Archivos modificados registrados en {log_path}")


✍️ Generando para: ./tests/test_core.py
✍️ Generando para: ./tests/test_providers.py
✍️ Generando para: ./cli/interactive_shell.py
✍️ Generando para: ./cli/run_agent.py
✍️ Generando para: ./logs/archivos_vaciados.txt
✍️ Generando para: ./agent/core.py
✍️ Generando para: ./agent/__init__.py
✍️ Generando para: ./agent/executor.py
✍️ Generando para: ./agent/context.py
✍️ Generando para: ./agent/objectives.py
✍️ Generando para: ./providers/__init__.py
✍️ Generando para: ./providers/fake_provider.py
✍️ Generando para: ./providers/openai_provider.py
✍️ Generando para: ./providers/claude_provider.py
✍️ Generando para: ./utils/__init__.py
✍️ Generando para: ./utils/logger.py
✅ Finalizado. Archivos modificados registrados en logs/archivos_generados.txt


import os
from pathlib import Path

# Ruta original (estructura funcional que se desea empujar)
origen = Path("estructura_original")
# Ruta destino (el proyecto principal)
destino = Path(".")

# Crear lista de comandos para limpiar el destino
comandos = []

# Recorrer la estructura original
for ruta in origen.rglob("*.py"):
    # Obtener la ruta relativa al proyecto original
    relativa = ruta.relative_to(origen)
    destino_final = destino / relativa

    # Crear directorio destino si no existe
    destino_final.parent.mkdir(parents=True, exist_ok=True)

    # Crear archivo vacío si no existe o limpiarlo si tiene contenido
    with open(destino_final, "w", encoding="utf-8") as f:
        f.write("")

# Mostrar resultado
f"✅ Archivos `.py` restaurados desde '{origen}' a '{destino}' como archivos vacíos."
